In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, auc, f1_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import shap
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Load datasets
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
ip_country = pd.read_csv('../data/IpAddress_to_Country.csv')
creditcard = pd.read_csv('../data/creditcard.csv')

In [10]:
# Task 1: Data Analysis and Preprocessing

# Handle Missing Values
fraud_data = fraud_data.dropna()  # Drop rows with missing values
creditcard = creditcard.dropna()

In [11]:
# Data Cleaning
fraud_data = fraud_data.drop_duplicates()
creditcard = creditcard.drop_duplicates()

In [12]:
# Correct Data Types
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
fraud_data['ip_address'] = fraud_data['ip_address'].astype(int)

In [13]:
# Merge Datasets for Geolocation Analysis
# Ensure IP addresses are integers
fraud_data['ip_address'] = fraud_data['ip_address'].astype(int)
ip_country['lower_bound_ip_address'] = ip_country['lower_bound_ip_address'].astype(int)
ip_country['upper_bound_ip_address'] = ip_country['upper_bound_ip_address'].astype(int)

# Sort datasets for merge_asof
fraud_data = fraud_data.sort_values('ip_address')
ip_country = ip_country.sort_values('lower_bound_ip_address')

# Perform merge_asof to match IPs to the closest lower_bound_ip_address
fraud_data['ip_key'] = fraud_data['ip_address']
merged = pd.merge_asof(
    fraud_data[['ip_key']],
    ip_country[['lower_bound_ip_address', 'upper_bound_ip_address', 'country']].assign(ip_key=ip_country['lower_bound_ip_address']),
    on='ip_key',
    direction='backward'
)

# Assign country where IP falls within range, else 'Unknown'
fraud_data['country'] = np.where(
    (merged['ip_key'] >= merged['lower_bound_ip_address']) & 
    (merged['ip_key'] <= merged['upper_bound_ip_address']),
    merged['country'],
    'Unknown'
)

# Clean up temporary column
fraud_data = fraud_data.drop('ip_key', axis=1)

In [14]:
# Feature Engineering for Fraud_Data
fraud_data['hour_of_day'] = fraud_data['purchase_time'].dt.hour
fraud_data['day_of_week'] = fraud_data['purchase_time'].dt.dayofweek
fraud_data['time_since_signup'] = (fraud_data['purchase_time'] - fraud_data['signup_time']).dt.total_seconds() / 3600


In [15]:
# Transaction Frequency and Velocity
fraud_data['transaction_count'] = fraud_data.groupby('user_id')['purchase_time'].transform('count')
fraud_data['velocity'] = fraud_data['purchase_value'] / (fraud_data['time_since_signup'] + 1)  # Avoid division by zero


In [16]:
# EDA: Class Distribution
print("Fraud_Data Class Distribution:")
print(fraud_data['class'].value_counts(normalize=True))
print("\nCreditcard Class Distribution:")
print(creditcard['Class'].value_counts(normalize=True))

Fraud_Data Class Distribution:
class
0    0.906354
1    0.093646
Name: proportion, dtype: float64

Creditcard Class Distribution:
Class
0    0.998333
1    0.001667
Name: proportion, dtype: float64


In [17]:
# Handle Class Imbalance with SMOTE (applied later during model training)

# Encode Categorical Features and Scale Numerical Features
categorical_features = ['source', 'browser', 'sex', 'country']
numerical_features = ['purchase_value', 'age', 'hour_of_day', 'day_of_week', 'time_since_signup', 'transaction_count', 'velocity']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])


In [18]:
# Task 2: Model Building and Training

# Prepare Fraud_Data
X_fraud = fraud_data.drop(['class', 'user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address'], axis=1)
y_fraud = fraud_data['class']
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud)


In [19]:
# Prepare Creditcard Data
X_credit = creditcard.drop('Class', axis=1)
y_credit = creditcard['Class']
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit)


In [20]:
# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_fraud_smote, y_train_fraud_smote = smote.fit_resample(preprocessor.fit_transform(X_train_fraud), y_train_fraud)
X_train_credit_smote, y_train_credit_smote = smote.fit_resample(X_train_credit, y_train_credit)


In [21]:
# Define Models
logreg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [22]:
# Train and Evaluate Models for Fraud_Data
pipeline_logreg_fraud = Pipeline([('preprocessor', preprocessor), ('classifier', logreg)])
pipeline_rf_fraud = Pipeline([('preprocessor', preprocessor), ('classifier', rf)])

pipeline_logreg_fraud.fit(X_train_fraud, y_train_fraud)
pipeline_rf_fraud.fit(X_train_fraud, y_train_fraud)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['purchase_value', 'age',
                                                   'hour_of_day', 'day_of_week',
                                                   'time_since_signup',
                                                   'transaction_count',
                                                   'velocity']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['source', 'browser', 'sex',
                                                   'country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [24]:
# Evaluate Models
def evaluate_model(model, X_test, y_test, dataset_name):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    auc_pr = auc(recall, precision)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    print(f"\n{dataset_name} Evaluation:")
    print(f"AUC-PR: {auc_pr:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    return auc_pr, f1

In [25]:
# Fraud_Data Evaluation
auc_pr_logreg_fraud, f1_logreg_fraud = evaluate_model(pipeline_logreg_fraud, X_test_fraud, y_test_fraud, "Fraud_Data Logistic Regression")
auc_pr_rf_fraud, f1_rf_fraud = evaluate_model(pipeline_rf_fraud, X_test_fraud, y_test_fraud, "Fraud_Data Random Forest")


c:\Users\mintesinot\week_8\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\mintesinot\week_8\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\mintesinot\week_8\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Fraud_Data Logistic Regression Evaluation:
AUC-PR: 0.6204
F1-Score: 0.6834
Confusion Matrix:
[[27358    35]
 [ 1343  1487]]


c:\Users\mintesinot\week_8\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Fraud_Data Random Forest Evaluation:
AUC-PR: 0.6166
F1-Score: 0.6878
Confusion Matrix:
[[27392     1]
 [ 1346  1484]]


In [26]:
# Train and Evaluate Models for Creditcard Data
logreg.fit(X_train_credit_smote, y_train_credit_smote)
rf.fit(X_train_credit_smote, y_train_credit_smote)


c:\Users\mintesinot\week_8\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForestClassifier(random_state=42)

In [27]:
# Creditcard Evaluation
auc_pr_logreg_credit, f1_logreg_credit = evaluate_model(logreg, X_test_credit, y_test_credit, "Creditcard Logistic Regression")
auc_pr_rf_credit, f1_rf_credit = evaluate_model(rf, X_test_credit, y_test_credit, "Creditcard Random Forest")



Creditcard Logistic Regression Evaluation:
AUC-PR: 0.7413
F1-Score: 0.2213
Confusion Matrix:
[[56087   564]
 [   13    82]]

Creditcard Random Forest Evaluation:
AUC-PR: 0.8110
F1-Score: 0.8249
Confusion Matrix:
[[56642     9]
 [   22    73]]


In [28]:
# Model Selection Justification
best_model = 'Random Forest' if (auc_pr_rf_fraud + auc_pr_rf_credit) > (auc_pr_logreg_fraud + auc_pr_logreg_credit) else 'Logistic Regression'
print(f"\nBest Model: {best_model}")
print("Justification: Random Forest typically performs better on imbalanced datasets due to its ability to capture complex patterns and interactions between features, which is critical for fraud detection.")



Best Model: Random Forest
Justification: Random Forest typically performs better on imbalanced datasets due to its ability to capture complex patterns and interactions between features, which is critical for fraud detection.


In [31]:
# Dense transformation
X_test_fraud_transformed = preprocessor.transform(X_test_fraud)
if hasattr(X_test_fraud_transformed, "toarray"):
    X_test_fraud_transformed = X_test_fraud_transformed.toarray()

# Correct feature names
feature_names = preprocessor.get_feature_names_out()

# SHAP Explainer
explainer = shap.TreeExplainer(pipeline_rf_fraud.named_steps['classifier'])
shap_values = explainer.shap_values(X_test_fraud_transformed)

# Plot SHAP
shap.summary_plot(shap_values[1], X_test_fraud_transformed, feature_names=feature_names)
plt.savefig('shap_summary_plot.png')


c:\Users\mintesinot\week_8\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.

In [ ]:
# Generate SHAP Summary Plot
feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
shap.summary_plot(shap_values[1], X_test_fraud_transformed, feature_names=feature_names)
plt.savefig('shap_summary_plot.png')

In [ ]:
# Interpretation
print("\nSHAP Summary Plot Interpretation:")
print("The SHAP summary plot shows the impact of each feature on the model's prediction of fraud. Features like 'time_since_signup' and 'velocity' are likely key drivers, indicating that rapid transactions after signup are strong indicators of fraud. Categorical features like 'country' and 'source' also contribute, suggesting geographic and acquisition channel patterns in fraudulent behavior.")